In [ ]:
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from metropolis import Metropolis

# Loading the dataset
met = Metropolis(
    "train",       # Name of the split we want to load (i.e. 'train', 'test', 'val')
    "metropolis/"  # Path to the dataset's root folder
)

# Navigating the dataset

In [ ]:
# Tables are available as attributes of `met`
pprint(met.instance[:3])
pprint(met.scene[:3])

In [ ]:
# We can quickly get table entries given their token using the `get()` function
sample = met.get(
    "sample",                 # Table name
    "tr1thGb4-HK8yPOzSZFHQQ"  # Token
)

pprint(sample)

In [ ]:
# Tables are cross-referenced using tokens, e.g. to get the scene a sample belongs to
scene = met.get("scene", sample["scene_token"])
pprint(scene)

# The `Metropolis` class decorates the tables with some useful reverse-indices
# e.g. all sample_data belonging to a sample are referenced in `sample["data"][{sensor_channel}]`
camera_left = met.get("sample_data", sample["data"]["CAM_LEFT"])
pprint(camera_left)

In [ ]:
# Samples belonging to the same scene are organized in a double linked list
# and can be easily traversed in a while loop
next_sample_token = scene["first_sample_token"]

while next_sample_token:
    next_sample = met.get("sample", next_sample_token)
    next_sample_token = next_sample["next_sample"]
    
    print(f"token: {next_sample['token']}, #2d annotations: {len(next_sample['anns_2d'])}")

In [ ]:
# Samples are also decorated with lists containing all annotations belonging to them
# `sample["anns"]` contains 3D annotation tokens, `sample["anns_2d"]` 2D annotation tokens
annotations = [met.get("sample_annotation", ann_token) for ann_token in sample["anns"]]

pprint(annotations[0])

# Annotations belonging to a sample can also be retrieved in a more structured way
# using the `get_sample_data()` method, more on this later
_, annotations_3d, _, _ = met.get_sample_data(sample["data"]["CAM_LEFT"])

pprint(annotations_3d[0])

In [ ]:
# Similar to samples and scenes, annotations belonging to the same instance are
# organized in a double linked list
instance = met.get("instance", annotations[0]["instance_token"])
next_annotation_token = instance["first_annotation_token"]

while next_annotation_token:
    next_annotation = met.get("sample_annotation_2d", next_annotation_token)
    next_annotation_token = next_annotation["next_sample_annotation"]
    
    print(f"token: {next_annotation['token']}, box: {next_annotation['bounding_box']}")

# Accessing the data

In [ ]:
CHANNEL = "CAM_FRONT"

# `get_sample_data()` collects all relevant information belonging to a sample data
# and returns it in a structured format
(
    raw_data_path,               # Path to the raw data file (e.g. an image)
    boxes,                       # List of visible 3D boxes
    boxes_2d,                    # List of visible 2D boxes
    intrinsics                   # For images only, camera intrinsic parameters
) = met.get_sample_data(
    sample["data"][CHANNEL],     # sample_data_token
    get_all_visible_boxes=True,  # Return all annotations vs. only those annotated on this sample
)

print(f"path: {raw_data_path}, #2d boxes: {len(boxes_2d)}, #3d boxes: {len(boxes)}")

# The raw data path can also be retrieved directly
raw_data_path = met.get_sample_data_path(sample["data"][CHANNEL])

Image.open(raw_data_path)

In [ ]:
# Project a point cloud into an image
(
    points,                       # 3xN numpy array with the project points (points[2, :] == 1.)
    depths,                       # N numpy array with the corresponding depth values
    _
) = met.map_pointcloud_to_image(
    sample["data"]["MVS"],        # sample_data_token of the point cloud
    sample["data"]["CAM_FRONT"],  # sample_data_token of the image
)

pprint(points[:2, :])
pprint(depths)

In [ ]:
# Point clouds are stored as npz files, and can be opened directly with numpy
points = np.load(met.get_sample_data_path(sample["data"]["MVS"]))
pprint(met.get_sample_data_path(sample["data"]["MVS"]))

pprint(points["points"])

In [ ]:
# Machine-generated panoptic masks can be retrieved for any image with
(
    meta,                        # Panoptic meta-data (see FORMAT.md for details)
    mask                         # Segmentation mask as numpy array of integers
) = met.get_panoptic_mask(
    sample["data"]["CAM_FRONT"]  # sample_data_token of the image
)

plt.imshow(mask)
plt.show()

pprint(meta)

# Visualizing the data

In [ ]:
# Rendering images with point clouds
fig = plt.figure()
ax = fig.add_subplot()

met.render_pointcloud_in_image(
    sample["token"],                  # sample_token
    pointsensor_channel="LIDAR_MX2",  # Name of the point cloud channel to visualize
    camera_channel="CAM_FRONT",       # Name of the camera channel to visualize
    ax=ax
)

ax.figure.set_size_inches(10, 10)
plt.show()

In [ ]:
# Rendering images with annotations
fig = plt.figure()
ax = fig.add_subplot()

met.render_sample_data(
    sample["data"]["CAM_FRONT"],     # sample_data_token of the image to visualize
    show_3d_boxes=True,              # True to render 3D boxes, False to render 2D boxes
    show_all_visible_3d_boxes=True,  # Render all visible 3D boxes vs. only those annotated on this sample
    ax=ax
)

ax.figure.set_size_inches(10, 10)
plt.show()

In [ ]:
# Rendering point clouds with annotations
fig = plt.figure()
ax = fig.add_subplot()

met.render_sample_data(
    sample["data"]["LIDAR_MX2"],     # sample_data_token of the point cloud to visualize
    show_all_visible_3d_boxes=True,  # Render all visible 3D boxes vs. only those annotated on this sample
    ax=ax
)

ax.figure.set_size_inches(10, 10)
plt.show()

In [ ]:
# Rendering images with panoptic masks
met.render_panoptic(
    sample["data"]["CAM_FRONT"],  # sample_data_token of the image to visualize
    out_path=None,
)

In [ ]:
# Rendering aerial view of the point clouds
# **NOTE**: this requires the GDAL library
fig = plt.figure()
ax = fig.add_subplot()

met.render_aerial_view(
    sample["data"]["MVS"],  # sample_data_token of the point cloud to visualize
    ax=ax
)

ax.figure.set_size_inches(10, 10)
plt.show()